In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet_v2 import ResNet101V2, ResNet152V2
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [2]:
import zipfile; zipfile.ZipFile('/content/drive/MyDrive/tb/tbdata.zip','r').extractall('TB DATA')

In [ ]:
datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input,  # Preprocess input for VGG16
    rotation_range=30,  # Rotate images by up to 30 degrees
    zoom_range=0.2,  # Random zoom
    width_shift_range=0.2,  # Randomly shift the image horizontally (by 20% of width)
    height_shift_range=0.2,  # Randomly shift the image vertically (by 20% of height)
    shear_range=0.2,  # Apply shear transformations
    brightness_range=[0.8, 1.2],  # Random brightness adjustments
    horizontal_flip=True,  # Flip images horizontally
    vertical_flip=False,  # Set to True if vertical flipping is needed
    fill_mode='nearest',  # How to fill pixels that are missing after a transformation
    validation_split=0.2  # 20% of the data for validation
)


In [ ]:

dir_path = '/kaggle/input/tb-classifi/Type of tb'


In [ ]:
base_models = {
    'VGG16': VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    'VGG19': VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    'ResNet50': ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    'ResNet101': ResNet101V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    'ResNet152': ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    'InceptionV3': InceptionV3(weights='imagenet', include_top=False, input_shape=(224,224,3)),
    'DenseNet201' : DenseNet201(weights='imagenet', include_top=False, input_shape=(224,224,3)),
    'EfficientNetB3' : EfficientNetB3(weights='imagenet', include_top=False, input_shape=(224,224,3)),
    'MobileNet' : MobileNet(weights='imagenet', include_top=False, input_shape=(224,224,3))
}


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.utils import class_weight

# Custom preprocessing function to adjust contrast
def custom_preprocessing(image):
    image = tf.keras.applications.vgg16.preprocess_input(image)
    image = tf.image.adjust_contrast(image, contrast_factor=0.5)  # Adjust contrast
    return image

# Define directory path
dir_path = '/kaggle/input/tb-classifi/Type of tb'

# Data Augmentation
datagen = ImageDataGenerator(
    preprocessing_function=custom_preprocessing,  # Custom preprocessing function
    rotation_range=30,  # Rotate images by up to 30 degrees
    zoom_range=0.2,  # Random zoom
    width_shift_range=0.2,  # Randomly shift the image horizontally (by 20% of width)
    height_shift_range=0.2,  # Randomly shift the image vertically (by 20% of height)
    shear_range=0.2,  # Apply shear transformations
    brightness_range=[0.8, 1.2],  # Random brightness adjustments
    horizontal_flip=True,  # Flip images horizontally
    vertical_flip=True,  # Added vertical flip
    fill_mode='nearest',  # How to fill pixels that are missing after a transformation
    validation_split=0.2  # 20% of the data for validation
)

# Train and Validation generators
train_generator = datagen.flow_from_directory(
    dir_path,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dir_path,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)

# Calculate class weights
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weight_dict = dict(enumerate(class_weights))

# List of base models to use (VGG16, VGG19, ResNet50, ResNet101)
base_models = {
    'VGG16': tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    'VGG19': tf.keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    'ResNet50': tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    'ResNet101': tf.keras.applications.ResNet101V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    'ResNet152': tf.keras.applications.ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    'InceptionV3': tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    'DenseNet201': tf.keras.applications.DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    'EfficientNetB3': tf.keras.applications.EfficientNetB3(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    'MobileNet': tf.keras.applications.MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
}


# Training loop for each model
for model_name, base_model in base_models.items():
    print(f"Training {model_name}")

    # Freeze base model layers initially
    for layer in base_model.layers:
        layer.trainable = False

    # Custom layers
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    predictions = layers.Dense(5, activation='softmax')(x)

    # Final model
    proposed_model = models.Model(inputs=base_model.input, outputs=predictions)
    proposed_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                           loss='categorical_crossentropy',
                           metrics=[tf.keras.metrics.CategoricalAccuracy(),
                                    tf.keras.metrics.Precision(),
                                    tf.keras.metrics.Recall()])

    # Callbacks
    # Callbacks
    check_point = ModelCheckpoint(f'{model_name}_best_model.keras', save_best_only=True)
    earlystopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3)


    # Initial Training
    history = proposed_model.fit(train_generator,
                                 epochs=20,
                                 validation_data=validation_generator,
                                 callbacks=[check_point, earlystopping, reduce_lr],
                                 class_weight=class_weight_dict)

    # Unfreeze last layers for fine-tuning
    for layer in base_model.layers[-10:]:
        layer.trainable = True

    # Fine-tuning the model with a lower learning rate
    proposed_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])

    history_fine_tuning = proposed_model.fit(train_generator,
                                             epochs=20,
                                             validation_data=validation_generator,
                                             callbacks=[check_point, earlystopping, reduce_lr],
                                             class_weight=class_weight_dict)


In [ ]:
import matplotlib.pyplot as plt

def plot_individual_model_accuracy(history, history_fine_tuning, model_name):
    acc = history.history['categorical_accuracy']
    val_acc = history.history['val_categorical_accuracy']

    acc_ft = history_fine_tuning.history['accuracy']
    val_acc_ft = history_fine_tuning.history['val_accuracy']

    plt.figure(figsize=(10, 6))
    plt.plot(acc + acc_ft, label='Training Accuracy')
    plt.plot(val_acc + val_acc_ft, label='Validation Accuracy')
    plt.title(f'{model_name} Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.show()

# Example usage for each model:
for model_name in base_models.keys():
    plot_individual_model_accuracy(history, history_fine_tuning, model_name)


In [ ]:
import matplotlib.pyplot as plt

def plot_individual_model_accuracy_and_loss(history, history_fine_tuning, model_name):
    # Accuracy
    acc = history.history['categorical_accuracy']
    val_acc = history.history['val_categorical_accuracy']
    acc_ft = history_fine_tuning.history['accuracy']
    val_acc_ft = history_fine_tuning.history['val_accuracy']

    # Loss
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    loss_ft = history_fine_tuning.history['loss']
    val_loss_ft = history_fine_tuning.history['val_loss']

    # Plotting Accuracy
    plt.figure(figsize=(10, 6))
    plt.plot(acc + acc_ft, label='Training Accuracy')
    plt.plot(val_acc + val_acc_ft, label='Validation Accuracy')
    plt.title(f'{model_name} Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.show()

    # Plotting Loss
    plt.figure(figsize=(10, 6))
    plt.plot(loss + loss_ft, label='Training Loss')
    plt.plot(val_loss + val_loss_ft, label='Validation Loss')
    plt.title(f'{model_name} Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.show()

# Example usage for each model:
for model_name in base_models.keys():
    plot_individual_model_accuracy_and_loss(history, history_fine_tuning, model_name)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Function to plot the confusion matrix
def plot_confusion_matrix(y_true, y_pred, class_names):
    # Generate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Normalize the confusion matrix
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(8, 6))
    sns.set(font_scale=1.2)
    
    # Use heatmap to visualize the confusion matrix
    sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)

    plt.title('Normalized Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.show()

# Example usage:
# Assuming `y_true` is the true labels and `y_pred` is the predicted labels.
# Replace with actual true and predicted labels from your dataset.
# For 5 classes: COVID, Extrapulmonary TB, Miliary TB, Normal, Pneumonia
class_names = ['COVID', 'Extrapulmonary TB', 'Miliary TB', 'Normal', 'Pneumonia']

# Example true and predicted labels
y_true = [0, 1, 2, 2, 4, 3, 1, 0]  # Actual class indices
y_pred = [0, 1, 2, 3, 4, 3, 1, 0]  # Predicted class indices

# Call the function
plot_confusion_matrix(y_true, y_pred, class_names)


In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

# Function to load the model, preprocess the image, and make predictions
def predict_image_class(model_path, img_path, target_size=(224, 224), class_names=None):
    # Load the trained model
    model = tf.keras.models.load_model(model_path)
    
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.vgg16.preprocess_input(img_array)  # Adjust if needed for different models

    # Make prediction
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)

    # Display the image and prediction
    plt.imshow(img)
    plt.axis('off')  # Turn off axis
    plt.title(f"Predicted Class: {class_names[predicted_class[0]]}" if class_names else f"Predicted Class ID: {predicted_class[0]}")
    plt.show()

# Example usage:
# Define the path to your image and model
img_path = "/kaggle/input/tb-classifi/Type of tb/Normal/NORMAL (1154).png"  # Change this to your test image path
model_path = "VGG16_best_model.keras"  # Change this to your model path
class_names = ['COVID', 'Extrapulmonary TB', 'Miliary TB', 'Normal', 'Pneumonia']  # List of class names

predict_image_class(model_path, img_path, target_size=(224, 224), class_names=class_names)
